In [30]:
import numpy as np
import pandas as pd
import keras
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.applications.vgg19 import decode_predictions
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
import cv2 as cv

display("Loading model....")
model = VGG19(weights='imagenet')
display("Model loaded.")

#display("Preprocessing Image")
#imageName = "ex1.png"
#image = cv.imread(imageName)
#imageHeight, imageWidth, imageChannels = image.shape
#display("Image " + str(imageName) + " is " + str(imageWidth) + "x" + str(imageHeight) + ".")
#image = cv.resize(image, (224,224)).astype(np.float32)
#image = image.transpose((2,0,1))
#image = np.expand_dims(image, axis=0)

#display("Predicting...")
#features = model.predict(image)
#display("Done!")
#label = decode_predictions(features)
#display(label)
#display(label[0])
#display(label[0][0])
#label = label[0]
#display(str(label[0][1]) + " (" + str(label[0][2]*100)+"%)")
#display(str(label[0]) + " (" + str(label[1]*100)+"%)")

'Loading model....'

'Model loaded.'

In [31]:
#Read in image
imageName = "ex1.png"
image = cv.imread(imageName)
imageHeight, imageWidth, imageChannels = image.shape

#Parameters to play around with
windowSize = 224 #REMEBER TO CHANGE THIS TO 128!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
numStacks = 4  #Nunmber of times to shrink the image. If this shrinks the image past
               #the point where the image size is less than the window size, the script will end
               #without any loss
zoomFactor = 2 # How much to initially enlarge image in order to zoom in on smaller boxes.
               # Smallest box size wikll be windowSize / zoomFactor, so a zoomfactor of 2 will
               # have a minimum size of 112 with a window size of 224.
scaleFactor = .9 #Determines how much smaller each subsequent scaling of the image is. Percentage
scaleX = 1 #How many times to apply a scale in the x direction only
scaleY = 1 #How many times to apply a scale in the y direction only
scaleXFactor = 1.5 #How much to stretch when scaling in the x direction (Percentage)
scaleYFactor = 1.5 #How much to stretch when scaling in the y direction
convStrideFactor = .5 #Percent of window size to use as convStride
convStride = int(windowSize * convStrideFactor)

class boxResult:
    x=0
    y=0
    height=0
    width=0
    classification=0
    confidence=0.0

def modelPredict(predictionImage, currentScale, currentScaleX=1, currentScaleY=1):
    predictionImage = predictionImage.astype(np.float32)
    predictionImage = np.expand_dims(predictionImage, axis=0)
    result = model.predict(predictionImage)

    ######################################################
    label = decode_predictions(result)[0]
    display(label)

    #This section makes sense of the VGG19 results and makes sense of them
    #As long as the highest weighted result ends up in temp.classification
    #and temp.confidence below, everything should still work when switching
    #models.
    ######################################################
    #Store results in an array
    #x, y, h, w, class, confidence
    temp = boxResult()
    temp.x = int(x / currentScaleX / currentScale)
    temp.y = int(y / currentScaleY / currentScale)
    temp.height = int(windowSize / currentScaleX / currentScale)
    temp.width = int(windowSize / currentScaleY / currentScale)
    temp.classification = label[1]################################
    temp.confididence = label[2]##################################
    return temp

#Image info
display("Image " + str(imageName) + " is " + str(imageWidth) + "x" + str(imageHeight) + ".")



#Scale up image to get zoomed in squares
image = cv.resize(image, None, fx=zoomFactor, fy=zoomFactor, interpolation = cv.INTER_CUBIC)
currentScale = zoomFactor
results = []

#Get frames then scale down
# number of times based on numStacks
for i in range (0,numStacks): 
    display("Stack #"+str(i))
    for y in range (0, int(imageHeight * currentScale) - windowSize):
        if y%convStride == 0:
            for x in range (0, int(imageWidth * currentScale) - windowSize):
                if x%convStride == 0:
                    #Perform classification on the baseline window
                    crop = image[y:y+windowSize, x:x+windowSize]
                    temp = modelPredict(crop, currentScale)
                    results.append(temp)
                    #Perform classification on any stretched windows
                    #X Stretches
                    for numStretches in range (0, scaleX):
                        display("Scaling x")
                        stretchedCrop = cv.resize(crop, None, fx=scaleXFactor*(1+numStretches), fy=1, interpolation = cv.INTER_CUBIC)
                        stretchedCropCropped = stretchedCrop[0:windowSize, 0:windowSize]
                        temp = modelPredict(stretchedCropCropped, currentScale, scaleXFactor*(1+numStretches))
                        results.append(temp)
                    for numStretches in range (0, scaleY):
                        display("Scaling y")
                        stretchedCrop = cv.resize(crop, None, fx=1, fy=scaleYFactor*(1+numStretches), interpolation = cv.INTER_CUBIC)
                        stretchedCropCropped = stretchedCrop[0:windowSize, 0:windowSize]
                        temp = modelPredict(stretchedCropCropped, currentScale, 1, scaleYFactor*(1+numStretches))
                        results.append(temp)
                    
    #Resize image, update currentScale
    image = cv.resize(image, None, fx=scaleFactor, fy=scaleFactor, interpolation = cv.INTER_AREA)
    currentScale *= scaleFactor
    
display("Done!")

'Image ex1.png is 602x404.'

'Stack #0'

[('n06359193', 'web_site', 0.24174668),
 ('n03532672', 'hook', 0.066418774),
 ('n03291819', 'envelope', 0.04223133),
 ('n04118776', 'rule', 0.031054791),
 ('n03126707', 'crane', 0.0205233)]

'Scaling x'

[('n06359193', 'web_site', 0.17338352),
 ('n03532672', 'hook', 0.083941676),
 ('n04141975', 'scale', 0.037448913),
 ('n03291819', 'envelope', 0.032005765),
 ('n03467068', 'guillotine', 0.03092017)]

...

[('n02104029', 'kuvasz', 0.35961547),
 ('n02090622', 'borzoi', 0.25297332),
 ('n02437616', 'llama', 0.12784803),
 ('n02111500', 'Great_Pyrenees', 0.0895819),
 ('n02100735', 'English_setter', 0.05760458)]

'Scaling x'

[('n02105505', 'komondor', 0.8636481),
 ('n02437616', 'llama', 0.027630625),
 ('n02104029', 'kuvasz', 0.0173623),
 ('n02091635', 'otterhound', 0.016447783),
 ('n02090721', 'Irish_wolfhound', 0.0101691885)]

'Scaling y'

KeyboardInterrupt: 

In [32]:
storageImage = cv.imread(imageName)
displayImage = storageImage.copy()
prevHeight = 0
lineColor = (255,0,0)
lineWidth = 2
for box in results:
    cv.rectangle(displayImage,(box.x,box.y),(box.x+box.width,box.y+box.height),lineColor,lineWidth)
    cv.imshow('Windows',displayImage)
    cv.waitKey(500)
    displayImage = storageImage.copy()
cv.destroyAllWindows()

In [35]:
#Variables to play with
confidenceThresh = 0.8 #Poin whether or not to prune a box from the list

#Create copy of results list to make playing with data easier
boxes = list(results)
             
#Pruning
for box in boxes:
    if box.confidence < confidenceThresh:
        boxes.remove(box)

#NMS
def nmsFast(boxes, overlapThreshold):
    #Ensure there are boxes
    if len(boxes) == 0:
        return []
    
    #Initialize list of picked boxes, and of coordinates
    x1 = np.array([])
    x2 = []
    y1 = []
    y2 = []
    pick = []
    
    #Get coordinates from bounding boxes and cast as floats
    for box in boxes:
        x1.append(float(box.x))
        x2.append(float(box.x+box.width))
        y1.append(float(box.y))
        y2.append(float(box.y+box.height))
    
    #Compute and store area of bounding boxes
    area = (x2-x1+1) * (y2-y1+1)
    
    #Sort boxes by bottom right coordinates
    idxs = np.argsort(y2)
    
    while len(idxs) > 0:
        last = len(idxs)-1
        i = idxs[last]
        pick.append(i)
        
        xx1 = np.maximum(x1[i],x1[idxs[:last]])
        xx2 = np.minimum(x2[i],x2[idxs[:last]])
        yy1 = np.maximum(y1[i],y1[idxs[:last]])
        yy2 = np.minimum(y2[i],y2[idxs[:last]])
        
        #compute width and height
        w = np.maximum(0, xx2-xx1+1)
        h = np.maximum(0, yy2-yy1+1)
        
        #compute overlap
        overlap = (w*h) / area[adxs[:last]]
        
        #delete indices that have overlap over overlapThreshold
        idxs = np.delete(idxs, np.concatenate(([last], np.where(overlap > overlapThresh)[0])))
    
    return boxes[pick]

In [37]:
nms_rez = nmsFast(results, .8)
print(len(nms_rez))






TypeError: unsupported operand type(s) for -: 'list' and 'list'